In [ ]:
%matplotlib nbagg
# Libraries
import scipy as sc
import numpy as np
import pymc3 as pm
import matplotlib.pyplot as plt
import matplotlib
from   IPython.display import Markdown, display
#
from   repropy import Context, Meta
import analysis
import analysis.plots as plot
from analysis.numass.meta import *
#
import unfolding as lib

In [ ]:
meta = Meta(
    dataset = dataset_2011 + Meta(
        drop_last=8,
    ),
    basis = Meta(
        oversample = 2,
    ),
    transmission = 'linear',
    omega = [
        Meta(kind='omega', deg=2, equalize=True),
    ],
)
ctx = Context(meta, store_dir='../store')
aa  = analysis.analytic.Analytic(ctx)

# Standard model

In [ ]:
model = pm.Model()
with model :
    # Smoothness parameter
    #alpha = pm.HalfFlat('alpha')
    # alpha = pm.Lognormal('alpha', mu=-4.86688525067991, sd=3.5e-1)
    # phi
    nPhi  = len(aa.basis.value())
    om    = aa.unfold.value().omegas[0].mat
    chol  = np.linalg.cholesky(np.linalg.inv(om))
    phi   = pm.MvNormal('phi',
                        mu=np.zeros(nPhi),
                        chol = chol/alpha,
                        shape=nPhi
                       )
    # f
    f     = pm.Normal('f', 
                      mu = pm.math.dot(aa.unfold.value().K, phi),
                      sd = aa.dat.value()['err'].values, 
                      shape=len(aa.dat.value()),
                      observed=aa.dat.value()['cnt'].values,
                     )

In [ ]:
#SEED = 383561
#np.random.seed(SEED)
with model :
    trace4 = pm.sample(draws=1000, cores=8)

In [ ]:
def plot_deconvolved_mcmc(res, only_mean=False, add=True) :
    "Plot deconvolved function and pointwise errors"
    vs  = np.linspace(aa.dat.value()['vs'].values[0], aa.dat.value()['vs'].values[-1], 500)
    phi = lib.PhiVec(np.average(res['phi'], axis=0), aa.basis.value(), np.cov(res['phi'], rowvar=False))
    ys = phi(vs)
    dy = np.asarray([phi.error(v) for v in vs])
    #
    if add :
        fig = plt.figure()
        plt.title('Deconvolution result')
    plt.plot(18700 - vs, ys, 'r')
    if not only_mean :
        plt.plot(18700 - aa.dat.value()['vs'].values, phi(aa.dat.value()['vs'].values), 'g.')
        plt.fill_between(18700 - vs, ys-dy, ys+dy, color='b', alpha=0.5)        
    plt.grid()
plot_deconvolved_mcmc(trace4)
#plot_deconvolved(r)

In [ ]:
plot.plot_correlations_matrix( np.cov(trace4['phi'], rowvar=False))
#plt.matshow(np.cov(trace['phi'], rowvar=False))
#plt.colorbar()

In [ ]:
pm.summary(trace4).round(3)

In [ ]:
pm.traceplot(trace4)
None

In [ ]:
pm.plot_posterior(trace4)
None

In [ ]:
plt.figure(figsize=(8, 60))
pm.forestplot(trace4)
None

In [ ]:
pm.autocorrplot(trace4)
None

In [ ]:
pm.densityplot(trace4)
None

# Positive-constrained

In [ ]:
modelP = pm.Model()
with modelP :
    # Smoothness parameter
    #alpha = pm.HalfFlat('alpha')
    #alpha = aa.alpha.value()
    alpha = pm.Lognormal('alpha', mu=-4.90, sd=3.4e-2)
    # phi
    nPhi = len(aa.basis.value())
    chol = np.linalg.cholesky( np.linalg.inv(aa.basis.value().omega(2)))
    low  = np.repeat(0, nPhi)
    phi  = pm.Bound(pm.MvNormal, lower=low)('phi',
                        mu=np.zeros(nPhi),
                        #cov = alpha*basis.value().omega(2),
                        chol = chol/alpha,
                        shape=nPhi
                       )
    # f
    f     = pm.Normal('f', 
                      mu = pm.math.dot(aa.unfold.value().K, phi),
                      sd = aa.dat.value()['err'].values, 
                      shape=len(aa.dat.value()),
                      observed=aa.dat.value()['cnt'].values,
                     )

In [ ]:
with modelP :
    # start = pm.find_MAP()
    traceP = pm.sample(draws=1000, cores=8, tune=1000)

In [ ]:
pm.summary(traceP).round(3)

In [ ]:
plot_deconvolved_mcmc(traceP)

In [ ]:
plot.plot_correlations_matrix( np.cov(traceP['phi'], rowvar=False))

In [ ]:
pm.traceplot(traceP)
None

In [ ]:
pm.plot_posterior(traceP)
None